In [1]:
import geopandas as gpd
import pandas as pd
from h3 import h3
import h3pandas
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
tqdm.pandas()

/tmp/ipykernel_8653/2039533087.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
%%time
# Read the input shapefile using GeoPandas
# Takes approximately 25s
blocks_raw = gpd.read_file("../data/blocks/tl_2020_23_tabblock20.zip")
blocks_raw.columns

CPU times: user 9.28 s, sys: 851 ms, total: 10.1 s
Wall time: 11 s


Index(['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20',
       'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20',
       'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'HOUSING20', 'POP20',
       'geometry'],
      dtype='object')

In [3]:
# Define the H3 resolution level
h3_resolution = 10

In [4]:
%%time
# Drop unneeded columns and get interior blocks using the h3pandas library
# Takes approximately 40s
blocks = blocks_raw[['GEOID20', 'geometry']].h3.polyfill(h3_resolution)
blocks.head(1)

CPU times: user 46.4 s, sys: 248 ms, total: 46.7 s
Wall time: 50.6 s


,GEOID20,geometry,h3_polyfill
0,230110205001020,"POLYGON ((-69.48805 44.50404, -69.48773 44.504...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580..."


In [5]:
# Add additional tiles near the edge of blocks using the h3 core library
def get_boundary_tiles(row):
    # Convert the polygon geometry to a list of coordinates
    try:
        coords = list(row['geometry'].exterior.coords)
    except AttributeError:
        coords = [coord for poly in row['geometry'].geoms for coord in list(poly.exterior.coords)]
    # Generate an H3 index for each coordinate using the H3 Core Library
    h3ids = [h3.geo_to_h3(lat, lng, h3_resolution) for lng, lat in coords]
    # Return a list of unique H3 tiles for the polygon
    return list(set(h3ids))

# Apply the get_h3ids function to the GeoDataFrame to generate H3 tiles for each polygon
blocks['h3_boundary'] = blocks.progress_apply(get_boundary_tiles, axis=1)
blocks.head(1)

  0%|          | 0/47138 [00:00<?, ?it/s]

,GEOID20,geometry,h3_polyfill,h3_boundary
0,230110205001020,"POLYGON ((-69.48805 44.50404, -69.48773 44.504...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580...","[8a2b1a735067fff, 8a2b1a735107fff, 8a2b1a09b11..."


In [6]:
blocks['h3ids'] = (blocks['h3_polyfill'] + blocks['h3_boundary']).apply(set).apply(list)
blocks.head(1)

,GEOID20,geometry,h3_polyfill,h3_boundary,h3ids
0,230110205001020,"POLYGON ((-69.48805 44.50404, -69.48773 44.504...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580...","[8a2b1a735067fff, 8a2b1a735107fff, 8a2b1a09b11...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580..."


In [7]:
%%time
# Define a function to calculate the fraction of the block area that overlaps each H3 tile
# Takes approximately 8 min (less for lower resolutions)
def calculate_area_fraction(row):
    # Get the list of H3 tiles for the Census Block
    h3ids = pd.DataFrame.from_records([{'fraction': 0, 'h3id': tile} for tile in row['h3ids']], index='h3id')
    # Add geometry for each tile
    h3ids = h3ids.h3.h3_to_geo_boundary().to_crs(3857)
    # Calculate the fraction of the intersection of the hexagon to the block
    h3ids['fraction'] = h3ids['geometry'].intersection(row['geometry']).area / row['geometry'].area
    return list(zip(h3ids.index.values, h3ids['fraction']))

# Apply the calculate_population_fraction function to the GeoDataFrame to calculate the population fraction for each H3 tile
blocks['h3_fraction']= blocks.to_crs(3857).progress_apply(calculate_area_fraction, axis=1)
blocks.head(1)

  0%|          | 0/47138 [00:00<?, ?it/s]

CPU times: user 13min 34s, sys: 11 s, total: 13min 45s
Wall time: 14min 42s


,GEOID20,geometry,h3_polyfill,h3_boundary,h3ids,h3_fraction
0,230110205001020,"POLYGON ((-69.48805 44.50404, -69.48773 44.504...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580...","[8a2b1a735067fff, 8a2b1a735107fff, 8a2b1a09b11...","[8a2b1a7358cffff, 8a2b1a7350dffff, 8a2b1a73580...","[(8a2b1a7358cffff, 0.004275039031168788), (8a2..."


In [8]:
blocks_explode = blocks[['GEOID20', 'h3_fraction']].explode('h3_fraction')
blocks_explode['h3id'] = blocks_explode['h3_fraction'].apply(lambda x: x[0])
blocks_explode['h3_fraction'] = blocks_explode['h3_fraction'].apply(lambda x: x[1])
blocks_explode.head(1)

,GEOID20,h3_fraction,h3id
0,230110205001020,0.004275,8a2b1a7358cffff


In [9]:
# Write the output GeoDataFrame to a CSV file
output_file = f"../data/blocks/tl_2020_23_tabblock20_h3_{h3_resolution}.csv"
blocks_explode.to_csv(output_file, index=False)